In [1]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
# Load training and test data
train_data = pd.read_csv("D:/IBA/ML/Competition/train.csv")  # Replace "train.csv" with your training file path
test_data = pd.read_csv("D:/IBA/ML/Competition/test.csv")    # Replace "test.csv" with your test file path

# Data preprocessing (you may need to customize this based on your dataset)
# For simplicity, we'll encode categorical features using LabelEncoder
label_encoder = LabelEncoder()
categorical_features = ["ethnicity", "gender", "icu_admit_source", "icu_stay_type", "icu_type", "apache_3j_bodysystem", "apache_2_bodysystem"]
for feature in categorical_features:
    train_data[feature] = label_encoder.fit_transform(train_data[feature])
    test_data[feature] = label_encoder.transform(test_data[feature])

# Replace missing values with means for variables with less than 30% missing values
train_data = train_data.fillna(train_data.mean())
test_data = test_data.fillna(test_data.mean())

# Split the training data into features and target variable
X = train_data.drop(columns=["RecordID", "hospital_death"])
y = train_data["hospital_death"]

# Train and make predictions using K-Nearest Neighbor with missing value handling
from sklearn.impute import SimpleImputer

# Create an imputer instance to handle missing values with a strategy (e.g., mean)
imputer = SimpleImputer(strategy="mean")

# Impute missing values in the feature matrix X
X_imputed = imputer.fit_transform(X)

# Train K-Nearest Neighbor on the imputed data with 250 neighbors
knn_model = KNeighborsClassifier(n_neighbors=7)
knn_model.fit(X_imputed, y)

# Impute missing values in the test data
test_data_imputed = imputer.transform(test_data.drop(columns=["RecordID"]))

# Make predictions using the trained model
knn_predictions = knn_model.predict_proba(test_data_imputed)
# Create submission DataFrames
submission_knn = pd.DataFrame({"RecordID": test_data["RecordID"], "hospital_death": knn_predictions[:, 1]})

# Save submission files to CSV
submission_knn.to_csv("submission_knn2 - 6.csv", index=False)